# **Retrievers**

## **What's Covered?**
- Introduction to Retrievers
    - Specify Top k
    - Specify Top k and Search Type
    - Maximum Marginal Relevance Retrieval
    - Similarity Score Threshold Retrieval
- Building an End-to-End RAG Chain
    - Step 1: Initialize the Chroma DB Connection
    - Step 2: Create a Retriever Object
    - Step 3: Initialize a Chat Prompt Template
    - Step 4: Initialize a Generator (i.e. Chat Model)
    - Step 5: Initialize a Output Parser
    - Step 6: Define a RAG Chain
    - Step 7: Invoke the Chain

## **Introduction to Retrievers**

There are times when we need to pass in Vector Stores as **retriever** objects, which can be easily done via a **as_retriever()** method call.

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

Retrievers accept a string query as input and return a list of Document's as output.

We can specify search type implemented by a vector store, like similarity and MMR (i.e. Maximum marginal relevance retrieval), to query the texts in the vector store.
- **Specify Top k**
```python
retriever = db_connection.as_retriever(search_kwargs={"k": 3})
```
- **Specify Top k and Search Type**
```python
retriever = db_connection.as_retriever(search_type="similarity", search_kwargs={"k": 3})
```
- **Maximum Marginal Relevance Retrieval**
```python
retriever = db_connection.as_retriever(search_type="mmr")
```
- **Similarity Score Threshold Retrieval**  
Apply a cutoff or a threshold such that any document which is below the cutoff is not returned.
```python
retriever = db_connection.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k": 3, "score_threshold": 0.5}
)
```

In [1]:
f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [2]:
# Step 1 - Initialize an embedding_model
# We are just loading OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Step 2 - Initialize a ChromaDB Connection
from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created
db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

In [4]:
db

In [5]:
# We can check the already existing values

print(len(db.get()["ids"]))

1004


In [6]:
# Converting CHROMA db connection to Retriever Object
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [7]:
query = "What is their on Julie vs Rachels List?"

results = retriever.invoke(query)

In [8]:
results

[Document(metadata={'source': 'data/subtitles/Friends_2x08.srt'}, page_content='126\n00:07:28,029 --> 00:07:29,621\nHe\'s gonna stay with Julie.\n\n127\n00:07:29,864 --> 00:07:31,957\nHe\'s gonna stay with her\nand she\'ll be:\n\n128\n00:07:32,233 --> 00:07:34,463\n"Hi, I\'m Julie. Ross picked me.\n\n129\n00:07:34,736 --> 00:07:38,797\nWe\'ll get married and have lots\nof kids and dig up stuff together!"\n\n130\n00:07:40,475 --> 00:07:43,137\nNo offense, but that\nsounds nothing like her.\n\n131\n00:07:46,080 --> 00:07:50,073\nWhat am I gonna do?\nThis is like a complete nightmare!\n\n132\n00:07:50,318 --> 00:07:54,448\nI know. This must be so hard.\n"Oh, no! Two women love me!\n\n133\n00:07:55,790 --> 00:07:59,055\nThey\'re both gorgeous,\nmy wallet\'s too small for my 50s...\n\n134\n00:07:59,260 --> 00:08:01,751\n...and my diamond shoes are too tight!"'),
 Document(metadata={'source': 'data/subtitles/Friends_2x08.srt'}, page_content='247\n00:15:09,082 --> 00:15:10,242\nNo! I\n\n248\n

## **Building an End-to-End RAG Chain**

**Step 1: Initialize the Chroma DB Connection**  
**Step 2: Create a Retriever Object**   
**Step 3: Initialize a Chat Prompt Template**  
**Step 4: Initialize a Generator (i.e. Chat Model)**  
**Step 5: Initialize a Output Parser**   
**Step 6: Define a RAG Chain**  
**Step 7: Invoke the Chain**

In [9]:
# Step 1: Initialize the Chroma DB Connection

from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created
db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

# We can check the already existing values
print(len(db.get()["ids"]))

1004


In [10]:
# Step 2: Create a Retriever Object 

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [11]:
# Step 3: Initialize a Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [12]:
# Step 4: Initialize a Generator (i.e. Chat Model)

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [13]:
# Step 5: Initialize a Output Parser

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [15]:
# Step 6: Define a RAG Chain

from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt_template | chat_model | parser

In [17]:
# Invoke the Chain

query = 'Who is Rachem?'

rag_chain.invoke(query)

'Rachem is not a person, it is a misinterpretation of the name Rachel. The speaker is questioning what the term "Rachem" means and if it is a term related to paleontology. They clarify that they wouldn\'t know because they are just a waitress.'

In [18]:
# Invoke the Chain

query = 'What is there on the List comparing Rachel and Julie?'

rag_chain.invoke(query)

'Rachel is described as being a waitress, while Julie is mentioned to have a lot in common with the speaker as they are both paleontologists.'